# Using Cuxfilter to Plot Coordinates on a Map of the United States
### A map of all United States households and tanks with a distance range slider
This visualization uses a the Cuxfilter library and GPUs to plot households and tanks on a dashboard, with range slider feature to show households that are a certain range of distance away from the nearest tank. There are also multiselect charts to view households within a certain age group and households within a specific distance category.

### Import statements

In [1]:
import geopandas as gpd
import numpy as np
import os

import cuxfilter
import cudf
from bokeh.palettes import Spectral11

import holoviews as hv
import pandas as pd

/opt/conda/envs/rapids/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Setting ```DATA_DIR```
In order to read in files from this repository, we must set ```DATA_DIR``` to be the data folder within this repository. This requires ```os.getcwd()``` to return the path to the processing notebook of this repository, so ```xxx/codeplus-celine-dcc-package/visualizations```, where ```xxx``` is the path to where you cloned this repository. If it is not, use ```os.chdir(path)``` to change the current working directory to ```xxx/codeplus-celine-dcc-package/visualizations``` before getting the current working directory in ```DATA_DIR = os.getcwd()```, where ```path``` is ```xxx/codeplus-celine-dcc-package/visualizations```.

In [2]:
DATA_DIR = os.getcwd()
DATA_DIR = DATA_DIR.replace('visualizations', 'data')
DATA_DIR

'/hpc/home/at341/ondemand/codeplus-celine-dcc-package/data'

### Reading in household and proximity data (pre-processed)
This is a preprocessed file with distance between households across the whole US (that have both children and eldery people) and tanks already calculated in miles. This dataframe also includes information on age group of the head of household, the latitude and longitudes of the tanks and households, distance category, and tank_type.

The ```lat_3857``` and ```lon_3857``` coordinates will be the points we plot on our cuxfilter dashboard, and the rest of the variables are used for the range slider and multiselect tools the user can interact with.

In [33]:
df = pd.read_parquet(DATA_DIR + '/hh_and_tank_risks.parquet')
df

,age_code,lat_3857,lon_3857,erqk_risks,swnd_risks,hrcn_risks,trnd_risks,cfld_risks,rfld_risks,avg_risk,distance_mi,distance_category,is_elderly
0,H,-1.372560e+07,6.030085e+06,23.971695,6.226360,-1.000000,11.394691,21.238291,6.071374,11.483735,46.055765,4,2
1,A,-1.374498e+07,5.330436e+06,2.593052,7.262484,-1.000000,9.914697,-1.000000,7.868933,4.606528,746.525231,4,2
2,I,-1.235813e+07,4.645040e+06,0.407726,8.066726,-1.000000,9.625419,-1.000000,6.357442,4.076219,829.214976,4,2
3,B,-8.951334e+06,6.179301e+06,3.312025,2.867939,-1.000000,10.280441,-1.000000,6.010181,3.745098,1300.845825,4,2
4,J,-8.215682e+06,5.191497e+06,3.862760,6.794995,3.156417,4.080984,-1.000000,5.920526,3.969280,158.427951,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
961384,None,-1.010035e+07,5.222881e+06,1.575536,17.648163,4.544047,21.537919,-1.000000,12.580429,9.647682,3200.000000,0,0
961385,None,-1.183249e+07,5.291041e+06,3.312025,2.867939,-1.000000,10.280441,-1.000000,6.010181,3.745098,3200.000000,0,0
961386,None,-9.971313e+06,4.384699e+06,17.807754,23.810359,8.253384,24.042775,-1.000000,18.432187,15.391077,3200.000000,0,0
961387,None,-7.944992e+06,5.135812e+06,9.400549,11.049468,5.819224,19.608082,7.130619,21.502062,12.418334,3200.000000,0,0


### Transforming to cuxfilter dataframe

This transforms the pandas dataframe into a cuDF dataframe, then from a cuDF dataframe into a Cuxfilter dataframe. This makes it possible to plot these dataframes using the Cuxfilter library.

In [34]:
cdf = cudf.DataFrame.from_pandas(df)

In [35]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf)

### Defining label maps
The Datashader plotting library that Cuxfilter uses to create our visualization through the use of Graphical Processing Units (GPUs) is optimized for working with large dataframes. Here, we're plotting over 12 million of them. However, an aspect of Datashader is that it only takes numerical inputs when creating the range slider and multiselect charts. This means that instead of being able to categorize each household by age group by labelling it with ```strings``` like ```'70-74```, we must label it numerically. Hence, our column ```age_group``` has numerical indicators. ```0``` indicates that the point is a tank, and the rest indicate that the point is a household and the head of household is of a specific age group seen in the ```label_age_group``` dictionary below. The same structure is true for our ```distance_category``` column. For the ```distance_category``` column, ```0``` indicates that the point is a tank, and the rest indicate that the point is a household which is a specific distance category away from a tank as seen in the ```label_distance_category``` dictionary. 

The label maps below associate each numerical value in our dataframe to a ```string``` label which is displayed on the range slider and multiselect. The ```colors``` list provides the hex codes for the coloring of each point in the map when it is displayed. 

Note: Using the synthetic data in this repo, we are only plotting around 1 million points. But using the original InfoUSA and AST datasets, this visualization would plot around 12 million of them.

In [36]:
label_age_group = {0: 'Tank', 1: '65+ (inferred)', 
             2: '65-69 (reported)', 3: '70-74', 
             4: '75+ (reported)'}

In [37]:
label_distance_category = {0: 'Tank', 1: '0.5 mi away from tank', 
             2: '1 mi away from tank', 3: '5 mi away from tank', 
             4: 'More than 5 mi away from tank'}

In [38]:
colors = list(Spectral11)
colors.reverse()

### Defining cuxfilter charts

This code defines the charts for our interactive dashboard. The ```points``` chart is the main map with households and tanks plotted. We are using latitude and longitude coordinates in the EPSG 3857 coordinate system, as it is the one used by the Cuxfilter library. The points for tanks and households are colored differently by setting the ```aggregate_col``` parameter as the ```distance_mi``` column in our dataframe. The ```aggregate_fn``` parameter, set to either ```max```, ```min``` or ```mean``` specifies which aggregation of the data to perform when coloring the points. 

This column has four categories, as described above, and the ```colors``` list has 11 different colors, used to color each point by its distance to the nearest tank as seen in the ```distance_mi``` column. 

The next three charts define the interactive range sliders and multiselects the user can interact with. In each of these lines, the ```.multi_select``` specifies that the chart is a multiselect chart, while the ```.range_slider``` specifies that the chart is a range slider. In each of these chart definitions, we specify the column name from our dataframe that the chart should pull from, and the label map Cuxfilter should use to create the chart. For example, the ```distance_category``` chart is a multiselect chart that pulls from the ```distance_category``` column. The options on that multiselect chart are labelled according to the ```label_map_distance``` label map.

In [39]:
points = cuxfilter.charts.scatter(x='lat_3857', y='lon_3857', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'mean', aggregate_col = 'distance_mi', tile_provider="CartoDark", title = 'Households with Elderly and Children in Proximity to Tanks',
                                  legend = True)

# age_group = cuxfilter.charts.multi_select('is_elderly', label_map=label_age_group)

distance_category = cuxfilter.charts.multi_select('distance_category', label_map=label_distance_category)

distance = cuxfilter.charts.range_slider('distance_mi')

Finally, we use the ```.dashboard``` method to put these charts together as an interactive dashboard for the user. We first specify the main charts which will be displayed in the ```layout``` we choose, then specify the charts that will be displayed on the dashboard's ```sidebar```.

In [40]:
# d = cux_df.dashboard([points], sidebar = [age_group, distance_category, distance])
d = cux_df.dashboard([points], sidebar = [distance_category, distance])

### Displaying interactive dashboard

Running the commands below displays the interactive dashboard. The user can use the multiselect charts to view specific subsets of points, and use ```ctrl``` click to view multiple catgories in one multiselect at a time. The user can also use the range slider chart to view households within a certain distance range from a storage tank.

Using these interactive tools creates a copy of the data every time a user interacts with them, which may cause a memory allocation error. You can run ```nvidia-smi``` to see how much memory you are using on the GPUs.

In [41]:
d.show()
d.app(sidebar_width=290) # run the dashboard within the notebook cell

Dashboard running at port 45741


Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=290)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=961389)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_category_multi_s..., width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, '0.5 mi away f...}, sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_mi_range_slider'..., width=400)
            [0] RangeSlider(end=3200.0, sizing_mode='stretch_both', start=0.199586293752788, value=(0.199586293752788, ..., value_end=3200.0, value_start=0.199586293752788)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Households with E..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')